# create_cmip6_globus_batch_files.ipynb
Create Globus batch files and scripts for ESGF CMIP6 data of interest.

B. Grandey, 2022.

In [1]:
! date

Thu Jan 20 13:18:06 +08 2022


In [2]:
import datetime
import json
import pandas as pd
import pathlib
import pyesgf
from pyesgf.search import SearchConnection
import re

# Print versions of packages
for module in [json, pd, pyesgf, re]:
    try:
        print('{}.__version__ = {}'.format(module.__name__, module.__version__))
    except AttributeError:
        pass

json.__version__ = 2.0.9
pandas.__version__ = 1.3.5
pyesgf.__version__ = 0.3.0
re.__version__ = 2.2.1


## Base paths

In [3]:
# Base path in which to save batch files and scripts
out_base = pathlib.Path('cmip6_globus_batch_files/').resolve()
# Base path in which to archive previously produced batch files and scripts
archive_base = pathlib.Path('archive/').resolve()
archive_base.mkdir(exist_ok=True)  # create directory if it does not yet exist
# Directory in which to save local cache for search connection
cache_dir = pathlib.Path('cache/').resolve()
cache_dir.mkdir(exist_ok=True)

## Archive previous contents of out_base

In [4]:
# If out_base exists, then archive it
if out_base.exists():
    now_str = datetime.datetime.now().strftime('%Y%m%d-%H%M')
    archive_dir = archive_base.joinpath(f'cmip6_globus_batch_files_{now_str}/')
    out_base.replace(archive_dir)
    cwd = pathlib.Path.cwd()
    print(f'Archived {out_base.relative_to(cwd)}/ to {archive_dir.relative_to(cwd)}/.')

Archived cmip6_globus_batch_files/ to archive/cmip6_globus_batch_files_20220120-1318/.


In [5]:
# Create new out_base
out_base.mkdir(exist_ok=True)

## Establish search connection

In [6]:
# Establish search connection
expire_after = datetime.timedelta(days=10)  # cache expiry
conn = SearchConnection('https://esgf-node.llnl.gov/esg-search',
                        distrib=True,
                        cache='cache/pyesgf_cache',  # enable local cache
                        expire_after=expire_after)
conn

## Identify suitable sources (models) and members (ripf variants)
Do this by finding source-member pairs that fulfil the following requirements:
1. Monthly data are available for at least one of 'zostoga', 'zos', and 'tas' variables.
2. Data are available for at least one of 'ssp585', 'ssp370', 'ssp245', and 'ssp126' experiments.
3. Data are available for both 'historical' and 'piControl' experiments.
4. The member is an 'r1i1' variant (e.g. 'r1i1p1f1', 'r1i1p5f2').

In [7]:
%%time
# Create dictionary to hold available experiments (list) for each source-member pair (tuple)
source_member_experiment_dict = dict()
# Perform initial search for datasets matching first two requirements above
ctx1 = conn.new_context(project='CMIP6',
                        variable=['zostoga', 'zos', 'tas'],
                        frequency='mon',
                        experiment_id=['ssp585', 'ssp370', 'ssp245', 'ssp126'])
# Loop over available sources
sources = sorted(ctx1.facet_counts['source_id'].keys())
for source_id in sources:
    # Constrain search to source, to identify available members
    ctx2 = ctx1.constrain(source_id=source_id)
    # Find r1i1 members (requirement #4)
    members = sorted(ctx2.facet_counts['member_id'].keys())
    members = [m for m in members if bool(re.match('r1i1', m))]
    # Loop over members
    for member_id in members:
        # Search for available experiments for this source-member pair
        ctx3 = conn.new_context(project='CMIP6',
                                variable=['zostoga', 'zos', 'tas'],
                                frequency='mon',
                                source_id=source_id,
                                member_id=member_id)
        experiments = sorted(ctx3.facet_counts['experiment_id'].keys())
        # Limit to experiments of interest
        experiments = [e for e in experiments if e in ['piControl', 'historical',
                                                       'ssp585', 'ssp370', 'ssp245', 'ssp126']]
        # Are data available for both the historical and piControl experiments?
        if ('historical' in experiments) and ('piControl' in experiments):
            # Save to dictionary
            source_member_experiment_dict[(source_id, member_id)] = experiments
            # Print
            print(f'{source_id} {member_id}: {experiments}')
# Summarise number of source-member pairs identified
print(f'{len(source_member_experiment_dict)} source-member pairs identified.')

ACCESS-CM2 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
ACCESS-ESM1-5 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
AWI-CM-1-1-MR r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
BCC-CSM2-MR r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
BCC-ESM1 r1i1p1f1: ['historical', 'piControl', 'ssp370']
CAMS-CSM1-0 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
CAS-ESM2-0 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
CESM2 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
CESM2-WACCM r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
CIESM r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp585']
CMCC-CM2-SR5 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
CMCC-ESM2 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp

## Function to find Globus URLs and write Globus batch files for a source-member pair

If one wishes to only find only one Globus URL for each unique NetCDF file, then it is faster to search for dataset results then skip datasets with an instance_id that has already been processed.
However, NetCDF files may be missed in practice: some Globus URLs may be inaccessible due to problems with endpoint accessibility, non-existent paths etc.
Therefore, it makes sense to build in some redundancy by including every Globus URL.

In light of these considerations, the function below performs a file search.

In [8]:
def write_globus_batch_files(source_id='ACCESS-CM2',
                             member_id='r1i1p1f1',
                             variables=['zostoga',],
                             frequency='mon',
                             experiments=['piControl', 'historical', 'ssp585', 'ssp370', 'ssp245', 'ssp126'],
                             conn=conn):
    """Find Globus URLs and write Globus batch files for a CMIP6 source-member pair.
    
    Keyword arguments:
      source_id -- string: ESGF source_id / model (default 'ACCESS-CM2')
      member_id -- string; ESGF member_id / ripf variant (default 'r1i1p1f1')
      variables -- list: variables of interest (default ['zostoga',])
      frequency -- string: time frequency of variable (default 'mon') 
      experiments -- list: experiment_id for experiments of interest
          (default ['piControl', 'historical', 'ssp585', 'ssp370', 'ssp245', 'ssp126'])
      conn -- pyesgf SearchConnection (default is a SearchConnection named 'conn')
    
    Returns:
      batch_fn_ep_dict -- dict: names of the batch files written (keys) with corresponding endpoint (values)
    """
    print(f'---- {source_id} {member_id} ----')
    # Create DataFrame to hold Globus info etc for search results
    globus_df = pd.DataFrame(columns=['variable', 'filename',
                                      'globus_url',  # URL (suitably unique to also use as index)
                                      'globus_ep', 'globus_path',  # Globus source endpoint and path
                                      'dest_path'])  # target path on destination endpoint
    # Create set to hold unique filenames for all variables (used for calculating filename coverage)
    unique_fns_all = set()
    # Loop over variables
    for v in variables:
        # File search context
        ctx1 = conn.new_context(project='CMIP6',
                                source_id=source_id,
                                member_id=member_id,
                                variable=v,
                                frequency=frequency,
                                experiment_id=experiments,
                                latest=True,
                                search_type='File')
        # Create set to hold unique filenames for this variable
        unique_fns_v = set()
        # Perform search and loop over file results
        file_results = ctx1.search()
        print(f'{v}: {len(file_results)} file results to process.')
        for f in file_results:
            # Is result marked as retracted? If so, then skip.
            if f.json['retracted']:
                continue
            # Add filename to unique_fns_v and unique_fns_all sets
            unique_fns_v.add(f.filename)
            unique_fns_all.add(f.filename)
            # Does Globus URL exist?
            globus_url = f.globus_url
            if globus_url:
                # Identify endpoint
                globus_ep = globus_url.split('/')[0]
                globus_ep = globus_ep.replace('globus:', '')
                if len(globus_ep) != 36:
                    print(f'globus_ep = "{globus_ep}" looks suspect. Skipping.')
                else:
                    # Path on endpoint
                    globus_path = globus_url.split(f'{globus_ep}/')[1]
                    # Target path on local endpoint (relative to $GCP_EP_CMIP6 environment variable)
                    instance_id = f.json['dataset_id'].split('|')[0]  # dataset's instance_id
                    dest_path = f'{v}/{source_id}_{member_id}/{instance_id}/{f.filename}'
                    # Update DataFrame
                    globus_df.at[globus_url] = {'variable': v, 'filename': f.filename,
                                                'globus_url': globus_url,
                                                'globus_ep': globus_ep, 'globus_path': globus_path,
                                                'dest_path': dest_path}
        # Print number of URLs found
        try:
            print(f'{v}: {globus_df["variable"].value_counts()[v]} Globus URLs saved.')
        except KeyError:
            print(f'{v}: No Globus URLs saved.')
        # Print number of unique filenames found for this variable
        print(f'{v}: {len(unique_fns_v)} unique filenames found (including non-Globus results).')
    # Dict to hold batch filenames (keys) and source endpoints (values)
    batch_fn_ep_dict = dict()
    # Loop over source endpoints
    for globus_ep in globus_df['globus_ep'].value_counts().index:
        # Select subset of data for this endpoint
        ep_df = globus_df[globus_df['globus_ep']==globus_ep]
        # Calculate filename coverage for this endpoint
        coverage = len(set(ep_df['filename'])) / len(unique_fns_all)
        # Get name of endpoint using Globus CLI
        ep_json = ! globus endpoint show -F json {globus_ep}
        ep_json = json.loads(''.join(ep_json))
        ep_name = ep_json['display_name']
        print(f'{ep_name}: {len(ep_df)} files in batch ({coverage:.0%} coverage).')
        # Label for transfer
        if len(variables) == 1:
            var_str = variables[0]
        elif len(variables) == 2:
            var_str = '-'.join(variables)
        else:
            var_str = f'{len(variables)}vars-inc-{variables[0]}'
        if len(experiments) == 1:
            exp_str = experiments[0]
        else:
            exp_str = f'{len(experiments)}exps'
        batch_label = f'{source_id}_{member_id}_{frequency}_{var_str}_{exp_str}_{globus_ep}'
        # Filename of batch file to write
        batch_fn = f'{batch_label}.txt'
        # Directory in which to write batch file
        batch_dir = out_base.joinpath(globus_ep)
        batch_dir.mkdir(exist_ok=True)
        # Write batch file
        with open(batch_dir.joinpath(batch_fn), 'w') as writer:
            writer.write(f'# Written by write_globus_batch_files() in create_cmip6_globus_batch_files.ipynb '
                         f'on {datetime.date.today()}.\n')
            writer.write(f'# Globus endpoint is {globus_ep} (Name: {ep_name}).\n')
            writer.write(f'# {len(ep_df)} files in batch ({coverage:.0%} coverage of filenames).\n')
            writer.write(f'# To activate source endpoint use Globus CLI:\n')
            writer.write(f'# globus endpoint activate --web {globus_ep}\n')
            writer.write(f'# To submit transfer use Globus CLI:\n')
            writer.write(f'# globus transfer {globus_ep} $GCP_EP_CMIP6 --batch {batch_fn} '
                         f'--preserve-mtime --fail-on-quota-errors --skip-source-errors --sync-level checksum '
                         f'--label "{batch_label}"\n')
            writer.write(f'# Replace $GCP_EP_CMIP6 with intended destination endpoint, including base path.\n')
            writer.write('\n')
            for i in ep_df.index:  # loop over rows of DataFrame
                globus_path = ep_df.loc[i]['globus_path']
                dest_path = ep_df.loc[i]['dest_path']
                writer.write(f'{globus_path} {dest_path}\n')
            print(f'Written {batch_fn} ({len(ep_df)}, {coverage:.0%})')
            batch_fn_ep_dict[batch_fn] = globus_ep
    return batch_fn_ep_dict

## Write batch files for source-member pairs identified above

In [9]:
# Create dictionary to hold all batch_fn_ep_dict results returned by write_globus_batch_files()
main_batch_fn_ep_dict = dict()

In [10]:
%%time
# Monthy data variables and experiments of interest
frequency = 'mon'
variables = ['zostoga', ]
experiments = ['piControl', 'historical', 'ssp585', 'ssp370', 'ssp245', 'ssp126']
# Loop over source-member pairs
for source_id, member_id in source_member_experiment_dict.keys():
    temp_dict = write_globus_batch_files(source_id=source_id,
                                         member_id=member_id,
                                         variables=variables,
                                         frequency=frequency,
                                         experiments=experiments)
    main_batch_fn_ep_dict.update(temp_dict)  # update dictionary with new filenames and endpoints

---- ACCESS-CM2 r1i1p1f1 ----
zostoga: 28 file results to process.
zostoga: 28 Globus URLs saved.
zostoga: 8 unique filenames found (including non-Globus results).
LLNL ESGF: 8 files in batch (100% coverage).
Written ACCESS-CM2_r1i1p1f1_mon_zostoga_6exps_415a6320-e49c-11e5-9798-22000b9da45e.txt (8, 100%)
CEDA ESGF DN1: 8 files in batch (100% coverage).
Written ACCESS-CM2_r1i1p1f1_mon_zostoga_6exps_ee3aa1a0-7e4c-11e6-afc4-22000b92c261.txt (8, 100%)
NCI ESGF: 8 files in batch (100% coverage).
Written ACCESS-CM2_r1i1p1f1_mon_zostoga_6exps_2058c7d6-a79f-11e6-9ad6-22000a1e3b52.txt (8, 100%)
DKRZ ESGF CMIP6: 4 files in batch (50% coverage).
Written ACCESS-CM2_r1i1p1f1_mon_zostoga_6exps_4981cd16-d651-11e6-9ccd-22000a1e3b52.txt (4, 50%)
---- ACCESS-ESM1-5 r1i1p1f1 ----
zostoga: 33 file results to process.
zostoga: 33 Globus URLs saved.
zostoga: 10 unique filenames found (including non-Globus results).
LLNL ESGF: 10 files in batch (100% coverage).
Written ACCESS-ESM1-5_r1i1p1f1_mon_zostoga_6exp

CEDA ESGF DN1: 4 files in batch (36% coverage).
Written CanESM5_r1i1p2f1_mon_zostoga_6exps_ee3aa1a0-7e4c-11e6-afc4-22000b92c261.txt (4, 36%)
---- CanESM5-CanOE r1i1p2f1 ----
zostoga: 19 file results to process.
zostoga: 10 Globus URLs saved.
zostoga: 9 unique filenames found (including non-Globus results).
LLNL ESGF: 5 files in batch (56% coverage).
Written CanESM5-CanOE_r1i1p2f1_mon_zostoga_6exps_415a6320-e49c-11e5-9798-22000b9da45e.txt (5, 56%)
DKRZ ESGF CMIP6: 5 files in batch (56% coverage).
Written CanESM5-CanOE_r1i1p2f1_mon_zostoga_6exps_4981cd16-d651-11e6-9ccd-22000a1e3b52.txt (5, 56%)
---- E3SM-1-1 r1i1p1f1 ----
zostoga: 0 file results to process.
zostoga: No Globus URLs saved.
zostoga: 0 unique filenames found (including non-Globus results).
---- EC-Earth3 r1i1p1f1 ----
zostoga: 3367 file results to process.
zostoga: 2357 Globus URLs saved.
zostoga: 1010 unique filenames found (including non-Globus results).
LLNL ESGF: 1010 files in batch (100% coverage).
Written EC-Earth3_r1i

LLNL ESGF: 7 files in batch (100% coverage).
Written INM-CM4-8_r1i1p1f1_mon_zostoga_6exps_415a6320-e49c-11e5-9798-22000b9da45e.txt (7, 100%)
CEDA ESGF DN1: 7 files in batch (100% coverage).
Written INM-CM4-8_r1i1p1f1_mon_zostoga_6exps_ee3aa1a0-7e4c-11e6-afc4-22000b92c261.txt (7, 100%)
DKRZ ESGF CMIP6: 7 files in batch (100% coverage).
Written INM-CM4-8_r1i1p1f1_mon_zostoga_6exps_4981cd16-d651-11e6-9ccd-22000a1e3b52.txt (7, 100%)
NCI ESGF: 5 files in batch (71% coverage).
Written INM-CM4-8_r1i1p1f1_mon_zostoga_6exps_2058c7d6-a79f-11e6-9ad6-22000a1e3b52.txt (5, 71%)
---- INM-CM5-0 r1i1p1f1 ----
zostoga: 44 file results to process.
zostoga: 44 Globus URLs saved.
zostoga: 14 unique filenames found (including non-Globus results).
CEDA ESGF DN1: 14 files in batch (100% coverage).
Written INM-CM5-0_r1i1p1f1_mon_zostoga_6exps_ee3aa1a0-7e4c-11e6-afc4-22000b92c261.txt (14, 100%)
DKRZ ESGF CMIP6: 14 files in batch (100% coverage).
Written INM-CM5-0_r1i1p1f1_mon_zostoga_6exps_4981cd16-d651-11e6-9c

LLNL ESGF: 22 files in batch (100% coverage).
Written UKESM1-0-LL_r1i1p1f2_mon_zostoga_6exps_415a6320-e49c-11e5-9798-22000b9da45e.txt (22, 100%)
CEDA ESGF DN1: 22 files in batch (100% coverage).
Written UKESM1-0-LL_r1i1p1f2_mon_zostoga_6exps_ee3aa1a0-7e4c-11e6-afc4-22000b92c261.txt (22, 100%)
DKRZ ESGF CMIP6: 22 files in batch (100% coverage).
Written UKESM1-0-LL_r1i1p1f2_mon_zostoga_6exps_4981cd16-d651-11e6-9ccd-22000a1e3b52.txt (22, 100%)
NCI ESGF: 4 files in batch (18% coverage).
Written UKESM1-0-LL_r1i1p1f2_mon_zostoga_6exps_2058c7d6-a79f-11e6-9ad6-22000a1e3b52.txt (4, 18%)
CPU times: user 12.2 s, sys: 1.73 s, total: 13.9 s
Wall time: 6min 1s


## Write batch files for specific combinations of source-member pair, variables, frequency, and experiments
Specific custom combinations can be added in this section.

Caution: to reduce the risk of two transfers trying to write the same local file simultaneously, these custom combinations should not overlap with the batch files produced above (e.g. if variable 'zostoga' is requested for all suitable source-member pairs above, then do not include 'zostoga' in the custom combinations below - unless it is for a source-member pair and/or experiment not covered above.)

In [11]:
%%time
# List containing tuples of custom combinations (source_id, member_id, variables, frequency, experiments)
comb_list = [('ACCESS-CM2', 'r1i1p1f1', ['thetao',], 'mon', ['ssp585',],),  # test 3D variable
             ]
# Produce batch file for each custom combination
for comb in comb_list:
    source_id, member_id, variables, frequency, experiments = comb
    temp_dict = write_globus_batch_files(source_id=source_id,
                                         member_id=member_id,
                                         variables=variables,
                                         frequency=frequency,
                                         experiments=experiments)
    main_batch_fn_ep_dict.update(temp_dict)  # update dictionary with new filenames and endpoints

---- ACCESS-CM2 r1i1p1f1 ----
thetao: 65 file results to process.
thetao: 53 Globus URLs saved.
thetao: 29 unique filenames found (including non-Globus results).
NCI ESGF: 29 files in batch (100% coverage).
Written ACCESS-CM2_r1i1p1f1_mon_thetao_ssp585_2058c7d6-a79f-11e6-9ad6-22000a1e3b52.txt (29, 100%)
LLNL ESGF: 12 files in batch (41% coverage).
Written ACCESS-CM2_r1i1p1f1_mon_thetao_ssp585_415a6320-e49c-11e5-9798-22000b9da45e.txt (12, 41%)
CEDA ESGF DN1: 12 files in batch (41% coverage).
Written ACCESS-CM2_r1i1p1f1_mon_thetao_ssp585_ee3aa1a0-7e4c-11e6-afc4-22000b92c261.txt (12, 41%)
CPU times: user 64 ms, sys: 35.1 ms, total: 99.1 ms
Wall time: 8.37 s


## Write main scripts for submission of Globus transfers
One main script for each source endpoint.

In [12]:
# Recommend that users do not run script if previously submitted Globus transfers are still running.
caution_str = ('Please do not run this script if previously submitted Globus transfers are still running '
               '(to reduce risk of two or more transfers trying to access same local file simultaneously, '
               'and in light of Globus limits).')

In [13]:
# Loop over source endpoints
eps_set = set(main_batch_fn_ep_dict.values())
for globus_ep in eps_set:
    # Get name of endpoint using Globus CLI
    ep_json = ! globus endpoint show -F json {globus_ep}
    ep_json = json.loads(''.join(ep_json))
    ep_name = ep_json['display_name']
    print(f'{globus_ep} (Name: {ep_name})')
    # Get list of batch filenames for this endpoint
    batch_fn_list = [fn for fn, ep in main_batch_fn_ep_dict.items() if ep==globus_ep]
    print(f'{len(batch_fn_list)} batch file(s).')
    # Script filename
    try:
        script_fn = f'0_main_{globus_ep}_{ep_name.replace(" ", "-")}.sh'
    except AttributeError:
        script_fn = f'0_main_{globus_ep}.sh'
    # Write script
    with open(out_base.joinpath(script_fn), 'w') as writer:
        writer.write(f'#!/usr/bin/zsh\n'  # use zsh
                     f'\n'
                     f'# Written by create_cmip6_globus_batch_files.ipynb on {datetime.date.today()}.\n'
                     f'# Globus endpoint is {globus_ep} (Name: {ep_name}).\n'
                     f'# {len(batch_fn_list)} batch file(s).\n'
                     f'# Caution:\n'
                     f'# {caution_str}\n'
                     f'# Environment variables:\n'
                     f'# $GCP_EP_CMIP6 should point to destination Globus endpoint, including desired path.\n'
                     f'# Usage:\n'
                     f'# zsh {script_fn}\n'
                     f'\n'
                     f'# Is endpoint activated?\n'
                     f'globus endpoint is-activated {globus_ep}\n'
                     f'if [ $? -ne 0 ]; then\n'
                     f'    echo "{globus_ep} is not activated. Please activate then re-run this script."\n'
                     f'    globus endpoint activate --web {globus_ep}\n'
                     f'    exit 1\n'
                     f'fi\n'
                     f'\n'
                     f'# Submit batch transfers\n'
                    )
        # Loop over batch files
        for batch_fn in batch_fn_list:
            batch_label = batch_fn.split('.')[0]
            writer.write(f'echo {batch_label}\n'
                         f'globus transfer {globus_ep} $GCP_EP_CMIP6 --batch {globus_ep}/{batch_fn} '
                         f'--preserve-mtime --fail-on-quota-errors --skip-source-errors --sync-level checksum '
                         f'--label "{batch_label}"\n'
                         f'sleep 1\n')
        writer.write('\n'
                     'exit 0\n')
        print(f'Written {script_fn}')

ee351394-6ac7-11e7-a9c0-22000bf2d287 (Name: IPSL UPMC ESGF)
1 batch file(s).
Written 0_main_ee351394-6ac7-11e7-a9c0-22000bf2d287_IPSL-UPMC-ESGF.sh
ee3aa1a0-7e4c-11e6-afc4-22000b92c261 (Name: CEDA ESGF DN1)
27 batch file(s).
Written 0_main_ee3aa1a0-7e4c-11e6-afc4-22000b92c261_CEDA-ESGF-DN1.sh
415a6320-e49c-11e5-9798-22000b9da45e (Name: LLNL ESGF)
35 batch file(s).
Written 0_main_415a6320-e49c-11e5-9798-22000b9da45e_LLNL-ESGF.sh
4981cd16-d651-11e6-9ccd-22000a1e3b52 (Name: DKRZ ESGF CMIP6)
31 batch file(s).
Written 0_main_4981cd16-d651-11e6-9ccd-22000a1e3b52_DKRZ-ESGF-CMIP6.sh
2058c7d6-a79f-11e6-9ad6-22000a1e3b52 (Name: NCI ESGF)
24 batch file(s).
Written 0_main_2058c7d6-a79f-11e6-9ad6-22000a1e3b52_NCI-ESGF.sh
4101e3a0-b7df-11eb-a16a-5fad80e6400b (Name: esgfcmcc#esgf-node2.cmcc.it)
2 batch file(s).
Written 0_main_4101e3a0-b7df-11eb-a16a-5fad80e6400b_esgfcmcc#esgf-node2.cmcc.it.sh
9805b3ba-d9bf-11e5-976c-22000b9da45e (Name: None)
2 batch file(s).
Written 0_main_9805b3ba-d9bf-11e5-976c-2200

In [14]:
! date

Thu Jan 20 13:24:36 +08 2022
